In [ ]:
import os
import sys
# os.environ["CUDA_VISIBLE_DEVICES"] = '-1'  # Без GPU
os.environ["CUDA_VISIBLE_DEVICES"] = '1'
os.environ['CUDA_DEVICE_ORDER'] = 'PCI_BUS_ID'
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

# Для работы в обоих контейнерах (dl_utils/docker и dl_utils/PreAnnotation):
sys.path += ['./src', './..']

In [ ]:
from ultralytics.models.sam import SAM3VideoSemanticPredictor

from utils import TimeIt, Beep
from ipy_utils import show_video, path2link
from cvat import (
    dir2unlabeled_tasks, tasks_auto_annottation, tasks2preview, subtask2xml,
    autofix_subtask, hide_skipped_objects_in_subtask, concat_dfs, get_column_ind,
    CVATPoints, split_df_to_tags_shapes_and_tracks,
)
from cvat_srv import CVATSRV, tqdm, CVATTask
from labels import LabelsConvertor
from ul_utils import UltralyticsModel

# Настройка параметров предразметки на SAM3:

In [ ]:
# Общие параметры работы модели:
overrides = dict(
    conf=0.25,  # Порог уверенности
    task='segment',
    mode='predict',
    imgsz=644,
    model='/home/user/models/sam3.pt',
    half=False,
    save=False,
    verbose=False,
)

# Словарь перехода Запрос -> метка:
prompts2labels = {
    'person': 'p',
    'car': 'c',
    'truck': 'c',
    'tent': 't',
    'building': 'h',
    'house': 'h',
    'boat': 'b'
}

# Минимальная длительность треков:
min_track_len = 10

# Сборка всех сырых данны для предразметки:

In [ ]:
# path = 'path/to/dir'               # Путь до папки
# path = 'path/to/file'              # Путь до файла
# path = ['list', 'to', 'files']     # Или список путей до файлов
path = './project/'

tasks = dir2unlabeled_tasks(path, desc='Индексация ресурсов')

# Предразметка:

In [ ]:
lc = LabelsConvertor(prompts2labels)

# Перебираем каждую задачу:
annotated_tasks = []
for task in tasks:
    annotated_task = []
    for old_df, file, true_frames in task:

        # Пересобираем модель, т.к. иначе при half=False будут проблемы:
        predictor = SAM3VideoSemanticPredictor(overrides=overrides)
        sam3 = UltralyticsModel(
            predictor,
            text=list(prompts2labels.keys()),
        )

        # Выполняем саму предразметку:
        df, file, true_frames = sam3.video2subtask(
            file,
            desc=f'Предразметка "{os.path.basename(file)}"'
        )

        # Выполняем подмену меток:
        with TimeIt('постобработку'):
            df = lc.apply2df(df)

            # Вносим изменения в разметку:
            for ind in range(len(df)):
                p = CVATPoints.from_dfrow(df.iloc[ind, :])

                # Переход от сегментов к прямоугольникам:
                p = p.asrectangle()

                # Добавляем к меткам треки:
                # p.attribs['label'] = f"{p.attribs['label']}: {p.attribs['track_id']}"

                df.iloc[ind, :] = p.to_dfrow().iloc[0, :]

            # Отбрасываем треки короче заданного числа кадров:
            tags_df, shapes_df, *track_dfs = split_df_to_tags_shapes_and_tracks(df)
            dfs = [tags_df, shapes_df]
            dfs += [
                track_df for track_df in track_dfs if len(track_df) >= min_track_len
            ]
            dfs.append(old_df)

            # Подготавливаем разметку для работы в CVAT и добавляем результат в список
            # задач:
            subtask = hide_skipped_objects_in_subtask(
                autofix_subtask(
                    (df, file, true_frames)
                )
            )

        annotated_task.append(subtask)
    annotated_tasks.append(annotated_task)

# Визуализация:
with TimeIt('сборку превью'), Beep():
    out = show_video(
        tasks2preview(
            annotated_tasks,
            fps=30,
            alpha=0.8
        ),
        size=320)
out

# Экспорт в CVAT:

In [ ]:
# Соединение с сервером:
cvat_srv = CVATSRV()
print('Соединение с', str(cvat_srv), 'установлено.')

In [ ]:
###################################################
# Определяем/создаём проект для сохранения задач: #
###################################################

undefined_label = 'None'

# Имя проекта в CVAT:
proj_name = 'Test Proj'

# Получаем множество теоретически возможных и фактически используемых меток:
labels = set(prompts2labels.values())
for task in annotated_tasks:
    for df, _, _ in task:
        labels |= set(df['label'])

# Если проект с зажданным именем существует:
if proj_name in cvat_srv.keys():

    # Берём его:
    proj = cvat_srv[proj_name]

    # Убеждаемся, что все необходимые метки в проекте уже есть:
    proj_labels = {label['name'] for label in proj.labels()}
    diff_labels = labels - proj_labels
    if diff_labels:
        raise ValueError(f'В проекте нет следующих меток: {diff_labels}')

    print('Используется имеющийся проект.')

else:
    proj = cvat_srv.new(proj_name, labels=labels)
    print('Создан новый проект.')


##################
# Сама выгрузка: #
##################


with Beep():
    xml_file = 'annotations.xml'
    for task in tqdm(annotated_tasks, desc='Выгрузка размеченных задач'):

        assert len(task) == 1

        df, file, true_frames = task[0]

        subtask2xml((df, file, true_frames), xml_file)

        if isinstance(file, str):
            name = os.path.basename(os.path.splitext(file)[0])
        else:
            name = os.path.basename(os.path.dirname(file[0]))

        if name in proj.keys():
            raise ValueError(f'В проекте уже есть задача с именем "{name}"!')

        proj.new(name, file, annotation_file=xml_file)

    print(f'В CVAT полностью выгружено {len(annotated_tasks)} задач(и).')